Diffusion pipeline are inherently a collection of diffusion models and schedulers that are partly independent from each other. This means that one is able to switch out parts of the pipeline to better customize a pipeline to one's use case. The best example of this is the Schedulers.

Whereas diffusion models usually simply define the forward pass from noise to a less noisy sample, schedulers define the whole denoising process, i.e.:
* How many denosing steps?
* Stochastic or deterministic?
* What algorithm to use to find the denoised sample

They can be quite complex and often define a trade-off between **denoising speed** and **denoising quality**. It is extremly difficult to measure quantiatively which shceduler works best for a given diffusion pipeline, so it is often recommended to simplu try out which works best.

### Load pipeline

In [ ]:
# Hard to lock to the specific version of dependencies https://github.com/huggingface/diffusers/issues/1255
!pip install --upgrade git+https://github.com/huggingface/diffusers.git

In [ ]:
import os, platform

torch_device = 'cpu'

if 'kaggle' in os.environ.get('KAGGLE_URL_BASE', 'localhost'):
    torch_device = 'cuda'
else:
    torch_device = 'mps' if platform.system() == 'Darwin' else 'cpu'

In [ ]:
torch_device

In [ ]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [ ]:
from diffusers import DiffusionPipeline
import torch


repo_id = "runwayml/stable-diffusion-v1-5"

#fp16 did not support by CPU inference sometimes
pipeline = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, device=torch_device)

#### Move to GPU

In [ ]:
pipeline.to(torch_device)

### Access the scheduler

The scheduler is always one of the components of the pipeline and is usually called "scheduler". So it can be accessed via the "scheduler" property.

In [ ]:
pipeline.scheduler

We can see that the scheduler is of type PNDMScheduler. Cool, now let's compare the scheduler in its performance to other schedulers. First we define a prompt on which we will test all the different schedulers:

In [ ]:
prompt = 'A photograph of an astronaut riding a horse on Mars, high resolution, high definition.'

Next, we create a generator from a random seed that will ensure that we can generate smimilar images as well as run the pipeline:

In [ ]:
generator = torch.Generator(device=torch_device).manual_seed(8)
image = pipeline(prompt, generator=generator).images[0]
image

### Changing the scheduler

Now we show how easy it is to change the scheduler or a pipeline. Every scheduler has a property SchedulerMixin. compatibles which defines all compatible schedulers. You can take a look at all available, compatible schedulers for the Stable Diffusion pipeline as follows:

In [ ]:
pipeline.scheduler.compatibles

### Comparing the input prompt with all other schedulers

To change the scheduler of the pipeline you can make use of the convenient ConfigMixin.config property in combination with the ConfigMixin.from_config() function, return a dictionary of the configuration of the scheduler:

In [ ]:
pipeline.scheduler.config

This configuration can then be used to instantiate a shcuduler of a different class that is compatible with the pipeline. Here, we change the scheduler to the DDIMScheduler:

In [ ]:
from diffusers import DDIMScheduler

pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)

Cool, now we can run the pipeline again to compare the genertion quality.

In [ ]:
generator = torch.Generator(device=torch_device).manual_seed(8)
image = pipeline(prompt, generator=generator).images[0]
image

### Compare schedulers

So far we have tried running the stable diffusion pipeline with two schedulers:
* PNDMScheduler
* DDIMScheduler

#### LMSDiscreteScheduler usually leads to **better results**

In [ ]:
from diffusers import LMSDiscreteScheduler

pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)

generator = torch.Generator(device=torch_device).manual_seed(8)
image= pipeline(prompt, generator=generator).images[0]
image

#### EulerDiscreteScheduler and EulerAncestralDiscreteScheduler can generate **high quality** results with as little as 30 steps

In [ ]:
from diffusers import EulerDiscreteScheduler

pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config)

generator = torch.Generator(device=torch_device).manual_seed(8)
image = pipeline(prompt, generator=generator, num_inference_steps=30).images[0]
image

In [ ]:
from diffusers import EulerAncestralDiscreteScheduler

pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config)

generator = torch.Generator(device=torch_device).manual_seed(8)
image = pipeline(prompt, generator=generator, num_inference_steps=30).images[0]
image

#### DPMSolverMultistepScheduler give arguably the **best speed/quality trade-off** and can be run with as little as 20 steps.

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

generator = torch.Generator(device=torch_device).manual_seed(8)
image = pipeline(prompt, generator=generator, num_inference_steps=30).images[0]
image

### Summary

As you can see most images look very similar and are arguably of very similar quality. It often reallt dependes on the specific use case which scheduler to choose. A good approach is always to run multiple shcedulers to compare results.